# Sarcasm Detection using Machine Learning

#### Goal of This File:

##### 1. Import Libraries, Helper Functions, and Constants ==> Data Sourcing and Munging ==> Utility Functions ==> Loading the Data ==> Merging all Individual Files

##### 2. Word Embeddings 

##### 3. Machine Learning Models ==> Small Model Test

##### 4. Conclusion ==> Results

## 1. Import Libraries, Helper Functions and Load Constants

In [3]:
!pip install pandas numpy nltk scikit-learn wordcloud\
seaborn gensim tensorflow imblearn xgboost matplotlib unrar pyunpack more-itertools patool keras-tqdm > /dev/null

The system cannot find the path specified.


In [4]:
VAST = False

if VAST:
    !sudo apt-get install unrar
    !sudo apt-get install rar

In [5]:
UTILS_URL = 'https://f005.backblazeb2.com/file/gp-support-files/archived_nlp_ai_utils.py'
UPDATING_VALUES_URL = 'https://f005.backblazeb2.com/file/gp-support-files/archived_updating_values.py'
ALL_LIBS_URL = 'https://f005.backblazeb2.com/file/gp-support-files/archived_all_libs_dl.py'
CHUNKS_URLS_FILE_URL = 'https://f005.backblazeb2.com/file/gp-support-files/chunks_urls.py'

UTILS_FILE_NAME = 'archived_nlp_ai_utils'
UPDATING_VALUES_FILE_NAME = 'archived_updating_values'
ALL_LIBS_FILE_NAME = 'archived_all_libs_dl'
CHUNKS_URLS_FILE_NAME = 'chunks_urls'

DEP_FILE_EXT = '.py'

In [6]:
import requests
import os

def get_dependencies(url,file_name,file_extension):
    if os.path.exists(file_name + file_extension):
        return print(file_name + " already exists.")
    else:
        print(f"downloading {file_name}...")
        r = requests.get(url)
        with open(file_name + file_extension, 'wb') as fd:
            for chunk in r.iter_content():
                fd.write(chunk)

In [7]:
get_dependencies(UTILS_URL,UTILS_FILE_NAME,DEP_FILE_EXT)
get_dependencies(UPDATING_VALUES_URL,UPDATING_VALUES_FILE_NAME,DEP_FILE_EXT)
get_dependencies(ALL_LIBS_URL,ALL_LIBS_FILE_NAME,DEP_FILE_EXT)
get_dependencies(CHUNKS_URLS_FILE_URL,CHUNKS_URLS_FILE_NAME,DEP_FILE_EXT)

archived_nlp_ai_utils already exists.
archived_updating_values already exists.
archived_all_libs_dl already exists.
chunks_urls already exists.


In [8]:
from archived_nlp_ai_utils import *
from chunks_urls import SD_CHUNKS_URLS
from archived_updating_values import *

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already u

In [9]:
TF_ENABLE_ONEDNN_OPTS = 0
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
DATA_URLS = SD_CHUNKS_URLS
GLOVE_URL = 'https://f005.backblazeb2.com/file/glove-embeddings-dims/glove.6B.100d.txt'
LIMIT = 10
TRAINED_MODELS_COUNT = TRAINED_MODELS
DATASET_PATH = "../datasets"
PICKLES_DIR = "../sd_pickle_files"
PREPROCESSED_CHUNKS_PATH = DATASET_PATH + "/preprocessed_sd_data_chunks/"
BASE_FILE_NAME = "sd_chunk_"
FILE_FORMAT = ".csv"
DATA_IN_CHUNK = 99744
if LIMIT == 10:
    ACTUAL_DATA_SHAPE = DATA_IN_CHUNK * LIMIT - 2
else:
    ACTUAL_DATA_SHAPE = DATA_IN_CHUNK * LIMIT
RANDOM_STATE = CONST_RANDOM_STATE
READY_DATASET_PATH = "../larger_dataset/ready_for_models/"

### 1.1 Data Sourcing and Munging

#### 1.1.1 Loading The Data

In [10]:
DATASET_PATH

'../datasets'

In [11]:
PREPROCESSED_CHUNKS_PATH

'../datasets/preprocessed_sd_data_chunks/'

In [12]:
if not os.path.exists(DATASET_PATH):
    os.mkdir(DATASET_PATH)
if not os.path.exists(PREPROCESSED_CHUNKS_PATH):
    os.mkdir(PREPROCESSED_CHUNKS_PATH)
get_chunks(DATA_URLS,LIMIT,1,BASE_FILE_NAME,PREPROCESSED_CHUNKS_PATH,FILE_FORMAT)

sd_chunk_1 already exists.
sd_chunk_2 already exists.
sd_chunk_3 already exists.
sd_chunk_4 already exists.
sd_chunk_5 already exists.
sd_chunk_6 already exists.
sd_chunk_7 already exists.
sd_chunk_8 already exists.
sd_chunk_9 already exists.
sd_chunk_10 already exists.


#### 1.1.2 Merging all Individual Files

In [13]:
#get all names of downloaded files
all_file_names = get_all_file_names(BASE_FILE_NAME,LIMIT)

In [14]:
#read all chunks into a list
list_dfs = read_chunks(all_file_names,PREPROCESSED_CHUNKS_PATH,FILE_FORMAT)

In [15]:
#concatenate all chunks into a singular df
df = group_up_chunks(list_dfs)

In [16]:
#check how much of the data was actually downloaded
percent_loaded = check_no_missing_data(df.shape[0],ACTUAL_DATA_SHAPE)
percent_loaded

100.0

In [17]:
df.reset_index(inplace = True)
df.drop(['index'],axis = 1,inplace = True)

In [18]:
X = df['text']
y = df['labels']

In [19]:
X = pd.Series([str(text) for text in X])

y = pd.Series([int(label) for label in y])

In [20]:
if not os.path.exists(PICKLES_DIR):
    os.mkdir(PICKLES_DIR)

## 2. Word Embeddings

### 2.1 Word2Vec

In [21]:
text_tokens = [nltk.word_tokenize(text) for text in X]

In [22]:
w2v_tokens = text_tokens

In [23]:
w2v_model = Word2Vec(sentences = w2v_tokens, vector_size = 100, window = 5, min_count=1, workers = 4)

In [24]:
X_W2V = np.zeros((len(text_tokens), w2v_model.vector_size))
for i, text in enumerate(w2v_tokens):
    for word in text:
        if word in w2v_model.wv:
            X_W2V[i] += w2v_model.wv[word]
    X_W2V[i] /= len(w2v_tokens)

### 2.2 TF-IDF

In [25]:
tfidf_tokens = text_tokens

In [26]:
dictionary = corpora.Dictionary(tfidf_tokens)

In [27]:
bow_corpus = [dictionary.doc2bow(token) for token in tfidf_tokens]

In [28]:
tfidf_model = TfidfModel(bow_corpus)

tfidf_embedding = tfidf_model[bow_corpus]

In [29]:
X_TFIDF = np.array([np.array(token)[:, 1] for token in tfidf_embedding])

C:\Users\user\AppData\Local\Temp\ipykernel_21620\3403743208.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_TFIDF = np.array([np.array(token)[:, 1] for token in tfidf_embedding])


### 2.3 FastText

In [30]:
fasttext_tokens = text_tokens

In [31]:
ft_model = FastText(sentences = fasttext_tokens, vector_size = 100, window = 5, min_count = 1, workers = 4)

In [32]:
X_FT = np.zeros((len(text_tokens), ft_model.vector_size))
for i, text in enumerate(fasttext_tokens):
    for word in text:
        if word in ft_model.wv:
            X_FT[i] += ft_model.wv[word]
    X_FT[i] /= len(text_tokens)

## 3. Machine Learning Models

In [ ]:
X_W2V = preprocessing.normalize(X_W2V)
X_TFIDF = preprocessing.normalize(X_TFIDF)
X_FT = preprocessing.normalize(X_FT)

### 3.1 Word2Vec Train-Validation-Test Split

In [ ]:
TRAIN_PERCENT = 0.8
VALID_TEST_PERCENT = 0.1
TRAIN_SIZE = int(ACTUAL_DATA_SHAPE * TRAIN_PERCENT)
VALID_TEST_SIZE = int(ACTUAL_DATA_SHAPE * VALID_TEST_PERCENT)
TOTAL_TEST_SIZE = TRAIN_SIZE + VALID_TEST_SIZE

In [33]:
train_set_w2v = X_W2V[:TRAIN_SIZE]
train_labels_w2v = y[:TRAIN_SIZE]
validation_set_w2v = X_W2V[TRAIN_SIZE:TOTAL_TEST_SIZE]
validation_labels_w2v = y[TRAIN_SIZE:TOTAL_TEST_SIZE]
test_set_w2v = X_W2V[TOTAL_TEST_SIZE:]
test_labels_w2v = y[TOTAL_TEST_SIZE:]

### 3.2 TF-IDF Train-Validation-Test Split

In [34]:
train_set_tfidf = X_TFIDF[:TRAIN_SIZE]
train_labels_tfidf = y[:TRAIN_SIZE]
validation_set_tfidf = X_TFIDF[TRAIN_SIZE:TOTAL_TEST_SIZE]
validation_labels_tfidf = y[TRAIN_SIZE:TOTAL_TEST_SIZE]
test_set_tfidf = X_TFIDF[TOTAL_TEST_SIZE:]
test_labels_tfidf = y[TOTAL_TEST_SIZE:]

### 3.3 FastText Train-Validation-Test Split

In [35]:
train_set_ft = X_FT[:TRAIN_SIZE]
train_labels_ft = y[:TRAIN_SIZE]
validation_set_ft = X_FT[TRAIN_SIZE:TOTAL_TEST_SIZE]
validation_labels_ft = y[TRAIN_SIZE:TOTAL_TEST_SIZE]
test_set_ft = X_FT[TOTAL_TEST_SIZE:]
test_labels_ft = y[TOTAL_TEST_SIZE:]

### 3.4 Naive Bayes

#### 3.4.1 Word2Vec 

In [ ]:
nb_clf_w2v = GaussianNB()
nb_clf_w2v.fit(train_set_w2v, train_labels_w2v)
validation_predictions_nb_w2v = nb_clf_w2v.predict(validation_set_w2v)
accuracy = metrics.accuracy_score(validation_labels_w2v,validation_predictions_nb_w2v)

#### 3.4.2 TF-IDF 

In [ ]:
nb_clf_tfidf = GaussianNB()
nb_clf_tfidf.fit(train_set_tfidf, train_labels_tfidf)
validation_predictions_nb_tfidf = nb_clf_w2v.predict(validation_set_tfidf)
accuracy = metrics.accuracy_score(validation_labels_tfidf,validation_predictions_nb_tfidf)

#### 3.4.3 FastText

In [ ]:
nb_clf_ft = GaussianNB()
nb_clf_ft.fit(train_set_ft, train_labels_ft)
validation_predictions_nb_ft = nb_clf_w2v.predict(validation_set_ft)
accuracy = metrics.accuracy_score(validation_labels_ft,validation_predictions_nb_ft)

### 3.5 SVM

#### 3.5.1 Word2Vec 

In [ ]:
svm_clf_w2v = SVC(kernel='rbf',random_state=RANDOM_STATE)
svm_clf_w2v.fit(train_set_w2v, train_labels_w2v)
validation_predictions_svm_w2v = svm_clf_w2v.predict(validation_set_w2v)
accuracy = metrics.accuracy_score(validation_labels_w2v,validation_predictions_svm_w2v)

#### 3.5.2 TF-IDF 

In [ ]:
svm_clf_tfidf = SVC(kernel='rbf',random_state=RANDOM_STATE)
svm_clf_tfidf.fit(train_set_tfidf, train_labels_tfidf)
validation_predictions_svm_tfidf = svm_clf_tfidf.predict(validation_set_tfidf)
accuracy = metrics.accuracy_score(validation_labels_tfidf,validation_predictions_svm_tfidf)

#### 3.5.3 FastText

In [ ]:
svm_clf_ft = SVC(kernel='rbf',random_state=RANDOM_STATE)
svm_clf_ft.fit(train_set_ft, train_labels_ft)
validation_predictions_svm_ft = svm_clf_ft.predict(validation_set_ft)
accuracy = metrics.accuracy_score(validation_labels_ft,validation_predictions_svm_ft)

### 3.6 Random Forest

#### 3.6.1 Word2Vec

In [36]:
rf_clf_w2v = RandomForestClassifier(n_estimators=50,random_state=RANDOM_STATE)
rf_clf_w2v.fit(train_set_w2v, train_labels_w2v)
validation_predictions_rf_w2v = rf_clf_w2v.predict(validation_set_w2v)
accuracy = metrics.accuracy_score(validation_labels_w2v,validation_predictions_rf_w2v)

#### 3.6.2 TF-IDF

In [ ]:
rf_clf_tfidf = RandomForestClassifier(n_estimators=50,random_state=RANDOM_STATE)
rf_clf_tfidf.fit(train_set_tfidf, train_labels_tfidf)
validation_predictions_rf_tfidf = rf_clf_tfidf.predict(validation_set_tfidf)
accuracy = metrics.accuracy_score(validation_labels_tfidf,validation_predictions_rf_tfidf)

#### 3.6.3 FastText

In [ ]:
rf_clf_ft = RandomForestClassifier(n_estimators=50,random_state=RANDOM_STATE)
rf_clf_ft.fit(train_set_ft, train_labels_ft)
validation_predictions_rf_ft = rf_clf_ft.predict(validation_set_ft)
accuracy = metrics.accuracy_score(validation_labels_ft,validation_predictions_rf_ft)

### 3.7 XGBoost

#### 3.7.1 Word2Vec

In [ ]:
train_labels_xgb_w2v,validation_labels_xgb_w2v = adjust_xgb_labels(train_labels_w2v,validation_labels_w2v)

In [ ]:
xgb_clf_w2v = XGBClassifier(random_state=RANDOM_STATE)
xgb_clf_w2v.fit(train_set_w2v, train_labels_xgb_w2v)
validation_predictions_xgb_w2v = xgb_clf_w2v.predict(validation_set_w2v)
accuracy = metrics.accuracy_score(validation_labels_xgb_w2v,validation_predictions_xgb_w2v)

#### 3.7.2 TF-IDF

In [ ]:
train_labels_xgb_tfidf,validation_labels_xgb_tfidf = adjust_xgb_labels(train_labels_tfidf,validation_labels_tfidf)

In [ ]:
xgb_clf_tfidf = XGBClassifier(random_state=RANDOM_STATE)
xgb_clf_tfidf.fit(train_set_tfidf, train_labels_xgb_tfidf)
validation_predictions_xgb_tfidf = xgb_clf_tfidf.predict(validation_set_tfidf)
accuracy = metrics.accuracy_score(validation_labels_xgb_tfidf,validation_predictions_xgb_tfidf)

#### 3.7.3 FastText

In [ ]:
train_labels_xgb_ft,validation_labels_xgb_ft = adjust_xgb_labels(train_labels_ft,validation_labels_ft)

In [ ]:
xgb_clf_ft = XGBClassifier(random_state=RANDOM_STATE)
xgb_clf_ft.fit(train_set_ft, train_labels_xgb_ft)
validation_predictions_xgb_ft = xgb_clf_ft.predict(validation_set_ft)
accuracy = metrics.accuracy_score(validation_labels_xgb_ft,validation_predictions_xgb_ft)

### 3.8 Small Model Test

In [ ]:
def test_model(sentence,embedding,clf):
    test_token = nltk.word_tokenize(sentence)
    test_vector = np.zeros(embedding.vector_size)
    for word in test_token:
        if word in embedding.wv:
            test_vector += embedding.wv[word]
    test_vector /= len(test_token)
    predicted_label = clf.predict([test_vector])[0]
    return predicted_label

In [ ]:
CLF_TAG = '_clf'
clf_name_flag = ['nb','svm','rf','_xgb']
word_embed_name_flag = ['_w2v','_tfidf','_ft']
FILE_EXT_PICKLE = '.pickle'
VALID_BASE_NAME = 'validation_predictions_'

#### 3.9 Save Models

In [ ]:
def save_models_pickles(PICKLES_DIR,clf_name_flag,word_embed_name_flag,FILE_EXT_PICKLE,VALID_BASE_NAME,clf_count,embed_count):
    for i in range(clf_count):
        for j in range(embed_count):
            saved_file_name = clf_name_flag[i] + word_embed_name_flag[j]
            saved_valid_key_name = VALID_BASE_NAME + clf_name_flag[i] + word_embed_name_flag[j]
            pickle_out = open(PICKLES_DIR + '/' + saved_file_name + FILE_EXT_PICKLE,'wb')
            pickle.dump(
                {saved_file_name:saved_file_name,\
                 saved_valid_key_name:saved_valid_key_name},pickle_out)
            pickle_out.close()
            print("saved file: " + clf_name_flag[i] + word_embed_name_flag[j] + FILE_EXT_PICKLE)

In [ ]:
def load_models_pickles(PICKLES_DIR,clf_name_flag,word_embed_name_flag,FILE_EXT_PICKLE,VALID_BASE_NAME,clf_count,embed_count):
    all_clfs_names = []
    all_valid_preds = []
    for i in range(clf_count):
        for j in range(embed_count):
            saved_file_name = clf_name_flag[i] + word_embed_name_flag[j]
            saved_valid_key_name = VALID_BASE_NAME + clf_name_flag[i] + word_embed_name_flag[j]
            if not os.path.exists(saved_file_name + FILE_EXT_PICKLE):
                print("file already exists.")
                continue
            else:
                file = pickle.load(open(PICKLES_DIR + '/' + saved_file_name + FILE_EXT_PICKLE,'rb'))
                all_clfs_names.append(file[saved_file_name])
                all_valid_preds.append(file[saved_valid_key_name])
                
    return all_clfs_names,all_valid_preds

In [ ]:
save_models_pickles(PICKLES_DIR,clf_name_flag,word_embed_name_flag,FILE_EXT_PICKLE,VALID_BASE_NAME,4,3)

In [ ]:
all_clfs,all_validation_predictions = load_models_pickles(PICKLES_DIR,clf_name_flag,word_embed_name_flag,FILE_EXT_PICKLE,VALID_BASE_NAME,4,3)

## 4. Conclusion

### 4.1 Results

In [ ]:
VALID_SET_VAR_BASE_NAME = 'validation_set'
VALID_LABELS_VAR_BASE_NAME = 'validation_labels'
X_BASE_NAME = 'X'

In [ ]:
for i in range(4):
    for j in range(3):
        clf = clf_name_flag[i] + '_clf' + word_embed_name_flag[j]
        show_metrics(clf,clf,VALID_SET_VAR_BASE_NAME + word_embed_name_flag[j],VALID_LABELS_VAR_BASE_NAME + word_embed_name_flag[j]\
                     ,VALID_BASE_NAME + word_embed_name_flag[j],X_BASE_NAME + (word_embed_name_flag[j]).upper(),y,5)